In [203]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
import os.path as osp
import glob
from skimage.transform import rescale, resize
from skimage.io import imsave, imread
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from collections import defaultdict

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")


In [204]:
class FaceLandmarksDataset():
    def __init__(self, root_dir, model_type="recognition", transform=None):
        self.data_list = glob.glob(osp.join(root_dir, '*.jpg'))
        labels = []
        for d in self.data_list:
            data = d.split("/")
            data = data[2].split("_")
            labels.append(data[0])
        self.labels = labels


    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = self.data_list[idx]
        
        # Image
        img = imread(image)
        img1 = resize(img, (224,224,3))
        image = torch.from_numpy(img1)
        # One Hot Labels
        label = self.labels[idx]

        for i in range(11):
            if int(label) == i:
                gt_label = i

        
        # Sample
        sample = {'image': image, 'gt_label': gt_label}
        return sample

In [205]:
Face_train = FaceLandmarksDataset(root_dir='./train_images/')

train_loader = DataLoader(Face_train, batch_size=1,
                        shuffle=False, num_workers=1)

Face_val = FaceLandmarksDataset(root_dir='./val_images/')
val_loader = DataLoader(Face_val, batch_size=1,
                        shuffle=False, num_workers=1)



In [209]:
import torch
import torch.nn as nn
import torch.nn.functional as F


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.layer_d0 = nn.Sequential(nn.Conv2d(3, 8, 7, padding=3, stride=1, bias=True),nn.ReLU())

        
        self.layer_d1 = nn.Sequential(nn.Conv2d(8, 16, 3, padding=1, stride=2, bias=True),
                                      nn.ReLU(), nn.Conv2d(16, 16, 3, padding=1, stride=1, bias=True))
        self.layer_d1s = nn.Conv2d(8, 16, 1, stride=2)
        self.relu_d1 = nn.ReLU()

        
        self.layer_d2 = nn.Sequential(nn.Conv2d(16, 32, 3, padding=1, stride=2, bias=True),
                                      nn.ReLU(), nn.Conv2d(32, 32, 3, padding=1, stride=1, bias=True))
        self.layer_d2s = nn.Conv2d(16, 32, 1, stride=2)
        self.relu_d2 = nn.ReLU()


        self.layer_d3 = nn.Sequential(nn.Conv2d(32, 64, 3, padding=1, stride=2, bias=True),
                                      nn.ReLU(), nn.Conv2d(64, 64, 3, padding=1, stride=1, bias=True))
        self.layer_d3s = nn.Conv2d(32, 64, 1, stride=2)
        self.relu_d3 = nn.ReLU()

        
#         self.layer_d4 = nn.Sequential(nn.Conv2d(64, 128, 3, padding=1, stride=2, bias=True),
#                                       nn.ReLU(), nn.Conv2d(128, 128, 3, padding=1, stride=1, bias=True))
#         self.layer_d4s = nn.Conv2d(64, 128, 1, stride=2)
#         self.relu_d4 = nn.ReLU()
        
        self.layer200 = nn.AvgPool2d(32,32)

        self.layer201 = nn.Sequential(nn.Linear(64, 11, bias=True))

    def forward(self, input_img):

        output = self.layer_d0(input_img)

        output_d1 = self.layer_d1(output)
        output_d1s = self.layer_d1s(output)
        output = output_d1 + output_d1s
        output = self.relu_d1(output)

        output_d2 = self.layer_d2(output)
        output_d2s = self.layer_d2s(output)
        output = output_d2 + output_d2s
        output = self.relu_d2(output)

        output_d3 = self.layer_d3(output)
        output_d3s = self.layer_d3s(output)
        output = output_d3 + output_d3s
        output = self.relu_d3(output)

#         output_d4 = self.layer_d4(output)
#         output_d4s = self.layer_d4s(output)
#         output = output_d4 + output_d4s
#         output = self.relu_d4(output)
        
        df4  = output
        
        gap_feats = self.layer200(output)
        gap_feats = gap_feats.view(gap_feats.shape[0], -1)
        probs = self.layer201(gap_feats)
        
 
#         df4 = torch.cat((df4),1)


        return probs


In [210]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Trainer(object):
    def __init__(self):
        self.global_step = 0
        self.epoch = 0
        
        self.model = Model()
#         print("ckcnlkcnkcs",self.model)
#         if torch.cuda.device_count() > 1:
#             print("Let's use", torch.cuda.device_count(), "GPUs!")
#             # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
#             device_ids = [i for i in range(torch.cuda.device_count())]
#             self.model = nn.DataParallel(self.model, device_ids)

        print(self.model)
        self.model.to(device)
        
        self.class_loss_fn1 = nn.CrossEntropyLoss()

        # OPTIMIZER
        no_wd = []
        wd = []
        print('Weight Decay applied to: ')



        for name, p in self.model.named_parameters():
            if not p.requires_grad:
                # No optimization for frozen params
                continue

            if 'bn' in name or 'bias' in name:
                no_wd.append(p)
            else:
                wd.append(p)
                print(name,)



        # Allow individual options
        self.optimizer = optim.Adam(
            [
                {'params': no_wd, 'weight_decay': 0.0},
                {'params': wd}
            ],
            lr= 2e-5,
            weight_decay=1e-5,
            amsgrad=False)

        self.lr_decay = optim.lr_scheduler.StepLR(self.optimizer, step_size=2,
                                                  gamma=0.1)

    def save_checkpoint(self, epoch):
        save_state = {
            'epoch': epoch,
            'global_step': self.global_step,
            'gcn_state_dict': self.model.state_dict(),
            'optimizer': self.optimizer.state_dict(),
            'lr_decay': self.lr_decay.state_dict()
        }

        save_name = os.path.join("./", 'checkpoints', 'epoch_plostp_%d_step%d.pth' \
                                 % (epoch, self.global_step))
        torch.save(save_state, save_name)
        print('Saved model')


    def loop(self):
        for epoch in range(self.epoch,100):
            self.epoch = epoch
            if self.epoch % 3 == 0 and self.epoch > 0:
                self.save_checkpoint(epoch)
            if self.epoch >= 90:
                break
            self.train(epoch)
            self.validate(epoch)

    def train(self, epoch):
        print('Starting training')
        self.model.train()
        losses = []
        accum = defaultdict(float)
        
        for step, data in enumerate(train_loader):

            img = data['image']
#             print(img)
#             img = torch.cat(img)
            img = img.view(-1, 224, 224, 3)
            img = torch.transpose(img, 1, 3)
            img = torch.transpose(img, 2, 3)
            img = img.float()
            
            gt_label = torch.tensor(data["gt_label"])
            
            probs = self.model(img.to(device))
            
            
            self.optimizer.zero_grad()
            
            class_loss = self.class_loss_fn1(probs, gt_label.to(device))
            
            loss_v = class_loss
            loss1 = class_loss
            loss_v.backward()
            
            self.optimizer.step()
            
            losses.append(loss1)

        avg_epoch_loss = 0.0
        for i in range(len(losses)):
            avg_epoch_loss += losses[i]

        avg_epoch_loss = avg_epoch_loss / len(losses)
        self.gcn_loss_sum_train = avg_epoch_loss

        print("Average Epoch %d loss is : %f" % (epoch, avg_epoch_loss))
        
    def validate(self,epoch):
        print('Validating')
        self.model.eval()
        losses = []
        with torch.no_grad():
            for step, data in enumerate(val_loader):
                img = data['image']
                img = img.view(-1, 224, 224, 3)
                img = torch.transpose(img, 1, 3)
                img = torch.transpose(img, 2, 3)
                img = img.float()
                
                gt_label = torch.tensor(data["gt_label"])
            
                probs = self.model(img.to(device))
                
                self.optimizer.zero_grad()
            
                class_loss = self.class_loss_fn1(probs, gt_label.to(device))

                loss = class_loss
                losses.append(loss)
            
            avg_epoch_loss = 0.0
            for i in range(len(losses)):
                avg_epoch_loss += losses[i]

            avg_epoch_loss = avg_epoch_loss / len(losses)
            self.gcn_loss_sum_train = avg_epoch_loss

            print("Average Epoch %d Validation loss is : %f" % (epoch, avg_epoch_loss))

In [ ]:
trainer = Trainer()
trainer.loop()

Model(
  (layer_d0): Sequential(
    (0): Conv2d(3, 8, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (1): ReLU()
  )
  (layer_d1): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (layer_d1s): Conv2d(8, 16, kernel_size=(1, 1), stride=(2, 2))
  (relu_d1): ReLU()
  (layer_d2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (layer_d2s): Conv2d(16, 32, kernel_size=(1, 1), stride=(2, 2))
  (relu_d2): ReLU()
  (layer_d3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (layer_d3s): Conv2d(32, 64, kernel_size=(1, 1), stride=(2, 2))
  (relu_d3): ReLU()
  (layer200): AvgPool2d(kernel_si

In [ ]:
def accuracy(output, target):
    global batch_size
    correct = 0
    for i in range(output.shape[0]):
        if output[i][1] >= 0.5:
            if target[i] == 1:
                correct += 1
        else:
            if target[i] == 0:
                correct += 1
    return correct
